In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# STEMI 의심 환자군 중 나머지 80% (FP 의심) 필터링

### 1. 데이터 로드 및 신규 환자 정보 확장
### 2. 신규 환자 정제, 컬럼 표준화 및 최종 병합

In [17]:
import pandas as pd
import os
import sqlite3
import numpy as np

# 1. 환경 설정 및 경로
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'
ed_base_path = os.path.join(base_path, 'mimic-iv-ed-2.2')
db_path = os.path.join(base_path, 'MIMIC4-hosp-icu.db')

# --- 입력 파일 ---
# (1) 기존 코호트 (ICD 확진)
original_cohort_path = os.path.join(base_path, 'cohort_ver40_reordered.csv')
# (2) 신규 발굴 환자 (ECG 텍스트)
hidden_candidates_path = os.path.join(base_path, 'hidden_stemi_candidates.csv')
# (3) ED 정보 (보강용)
edstays_csv_path = os.path.join(ed_base_path, 'edstays.csv')

# --- 출력 파일 ---
save_path = os.path.join(base_path, 'cohort_ver21_expanded_STEMI(2).csv') # 최종 저장 파일

print("코호트 병합, 확장, 정제 작업 시작")

try:
    # 2. 데이터 로드
    print("데이터 로드 중...")

    # (1) 기존 코호트 (기준)
    df_old = pd.read_csv(original_cohort_path)
    df_old['cohort_source'] = 'ICD_Confirmed'
    print(f"  - 기존 코호트(ICD) 로드 완료: {len(df_old)}건")

    # (2) 신규 환자 (ECG 텍스트 기반)
    df_new_raw = pd.read_csv(hidden_candidates_path)
    print(f"  - 신규 후보군(ECG) 로드 완료: {len(df_new_raw)}건")

    # (3) 보강용 데이터 (DB 및 CSV)
    df_edstays = pd.read_csv(edstays_csv_path,
                             dtype={'subject_id': int, 'hadm_id': 'Int64', 'stay_id': int},
                             parse_dates=['intime', 'outtime'])

    conn = sqlite3.connect(db_path)
    df_patients = pd.read_sql_query("SELECT subject_id, gender, anchor_age FROM patients", conn)
    df_admissions = pd.read_sql_query("SELECT hadm_id, race FROM admissions", conn)
    conn.close()
    print("  - 보강용 데이터(ED, Patients, Admissions) 로드 완료")


    # 3. 신규 환자 정보 보강 (Merge Chain)
    # (df_new_raw는 stay_id를 가지고 있으므로, stay_id를 key로 사용)
    print("신규 환자 정보 보강 중 (중복 컬럼 방지)...")

    # (A) ED 정보 (hadm_id, disposition, intime) 추가
    # df_new_raw의 stay_id를 기준으로 edstays의 정보를 붙임
    df_new_enriched = pd.merge(
        df_new_raw,
        df_edstays[['stay_id', 'hadm_id', 'disposition', 'intime', 'outtime']],
        on='stay_id',
        how='left' # 신규 환자(df_new_raw)는 모두 유지
    )

    # (B) 환자 정보 (gender, anchor_age) 추가
    df_new_enriched = pd.merge(df_new_enriched, df_patients, on='subject_id', how='left')

    # (C) 인종 정보 (race) 추가
    # (A)에서 얻은 hadm_id를 사용
    df_new_enriched = pd.merge(df_new_enriched, df_admissions, on='hadm_id', how='left')

    print(f"  - 신규 환자 정보 보강 완료 (Shape: {df_new_enriched.shape})")


    # 4. 신규 환자 정제 (Filtering)
    print("신규 환자 정제 (필터링) 시작...")

    # (A) 텍스트 부정어(Negation) 제거
    df_new_enriched['interp_lower'] = df_new_enriched['interpretation'].fillna('').astype(str).str.lower()
    negation_keywords = [
        'no st elevation', 'rule out', 'ruled out', 'negative for',
        'history of', 'old inferior', 'old anterior', 'prior'
    ]
    mask_negation = df_new_enriched['interp_lower'].str.contains('|'.join(negation_keywords), na=False)
    df_new_filtered = df_new_enriched[~mask_negation].copy()
    print(f"   -> 텍스트 부정어 제외: -{mask_negation.sum()}명")

    # (B) 응급실 퇴실 경로(Disposition) 확인
    mask_home = df_new_filtered['disposition'] == 'HOME'
    df_new_refined = df_new_filtered[~mask_home].copy()
    print(f"   -> 응급실 귀가(HOME) 제외: -{mask_home.sum()}명")

    df_new_refined['cohort_source'] = 'ECG_Text_Mining'
    print(f"  - 정제된 신규 환자: {len(df_new_refined)}명")


    # 5. 컬럼 표준화 및 최종 병합
    print("컬럼 표준화 및 최종 병합 중...")

    # (A) 기존 코호트(df_old)의 컬럼을 기준으로 삼음
    target_columns = df_old.columns.tolist()

    # (B) 신규 환자 데이터에 없는 '기존 컬럼'은 NaN으로 추가
    for col in target_columns:
        if col not in df_new_refined.columns:
            df_new_refined[col] = np.nan

    # (C) 기존 코호트에 없는 '신규 컬럼' (interpretation 등)은 NaN으로 추가
    extra_cols = [col for col in df_new_refined.columns if col not in target_columns]
    for col in extra_cols:
        df_old[col] = np.nan

    # (D) 최종 컬럼 순서 정의 (기존 + 신규)
    final_column_order = target_columns + extra_cols

    # 순서 정렬
    df_old_aligned = df_old[final_column_order]
    df_new_aligned = df_new_refined[final_column_order]

    # (E) 최종 병합
    final_cohort = pd.concat([df_old_aligned, df_new_aligned], ignore_index=True)

    # 6. 결과 저장
    final_cohort.to_csv(save_path, index=False, encoding='utf-8-sig')

    print("\n" + "-"*30)
    print("최종 병합 및 정제 결과")
    print(f"1. 기존 코호트: {len(df_old_aligned)}명")
    print(f"2. 정제된 신규 환자: {len(df_new_aligned)}명")
    print(f"3. 최종 코호트 크기: {len(final_cohort)}명")
    print(f"저장 경로: {save_path}")
    print("-"*30)

    print("\n[최종 결측치 확인 (주요 컬럼)]")
    print(final_cohort[['subject_id', 'gender', 'race', 'disposition']].isnull().sum())

    print("\n[데이터 샘플 (컬럼 구조 확인)]")
    print(final_cohort[['subject_id', 'cohort_source', 'gender', 'race', 'interpretation']].head())
    print(final_cohort[['subject_id', 'cohort_source', 'gender', 'race', 'interpretation']].tail())

except Exception as e:
    print(f"오류 발생: {e}")

코호트 병합, 확장, 정제 작업 시작
데이터 로드 중...
  - 기존 코호트(ICD) 로드 완료: 836건
  - 신규 후보군(ECG) 로드 완료: 2153건
  - 보강용 데이터(ED, Patients, Admissions) 로드 완료
신규 환자 정보 보강 중 (중복 컬럼 방지)...
  - 신규 환자 정보 보강 완료 (Shape: (2153, 22))
신규 환자 정제 (필터링) 시작...
   -> 텍스트 부정어 제외: -4명
   -> 응급실 귀가(HOME) 제외: -1002명
  - 정제된 신규 환자: 1147명
컬럼 표준화 및 최종 병합 중...

------------------------------
최종 병합 및 정제 결과
1. 기존 코호트: 836명
2. 정제된 신규 환자: 1147명
3. 최종 코호트 크기: 1983명
저장 경로: /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/cohort_ver21_expanded_STEMI(2).csv
------------------------------

[최종 결측치 확인 (주요 컬럼)]
subject_id      0
gender          0
race           53
disposition     0
dtype: int64

[데이터 샘플 (컬럼 구조 확인)]
   subject_id  cohort_source gender                      race interpretation
0    10010058  ICD_Confirmed      M  ASIAN - SOUTH EAST ASIAN            NaN
1    10012438  ICD_Confirmed      M                     WHITE            NaN
2    10013310  ICD_Confirmed      F             BLACK/AFRICAN            NaN
3    10013310  ICD_C

## 3. 신규 STEMI 추출 과정에서 생성된 불필요한 column 삭제

In [13]:
import pandas as pd
import os
from google.colab import drive

# 1. 드라이브 마운트 (새로운 셀에서 실행 시 필요)
try:
    drive.mount('/content/drive')
except:
    pass # 이미 마운트된 경우

# 2. 경로 설정
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'
cohort_path = os.path.join(base_path, 'cohort_ver21_expanded_STEMI(2).csv')

print("파일을 로드하여 24번째 열(인덱스 23)의 이름을 확인합니다.")

try:
    # 3. 데이터 로드
    if not os.path.exists(cohort_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {cohort_path}")

    df_cohort = pd.read_csv(cohort_path)

    # 4. 24번째 열 이름 확인 (0부터 시작하므로 인덱스는 23)
    if len(df_cohort.columns) > 23:
        column_index_23 = df_cohort.columns[23]
        print("\n" + "="*40)
        print(f"24번째 열(인덱스 23)의 이름은:")
        print(f"->  {column_index_23}")
        print("="*40)
    else:
        print(f"\n오류: 이 파일에는 총 {len(df_cohort.columns)}개의 열만 존재하여 24번째 열을 확인할 수 없습니다.")

except Exception as e:
    print(f"오류 발생: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
파일을 로드하여 24번째 열(인덱스 23)의 이름을 확인합니다.

24번째 열(인덱스 23)의 이름은:
->  study_id


# 신규 STEMI (1983명) EDA

### 1. 폰트 설치

In [14]:
# [Cell 1] 폰트 설치 (세션 재시작 필요)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

print("폰트 설치가 완료되었습니다.")
print("반드시 상단 메뉴의 [런타임] -> [세션 다시 시작]을 클릭한 뒤, [Cell 2]부터 실행하세요!")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (11.1 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121703 files and direc

### 2. 불필요한 컬럼 삭제
- 런타임 재시작 후 아래 셀 실행

In [1]:
import pandas as pd
import os
from google.colab import drive

# 0. 드라이브 마운트 (세션 재시작 후 필수)
try:
    drive.mount('/content/drive')
except:
    pass # 이미 마운트된 경우

# 1. 경로 설정
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'
# 24번째 열부터 삭제할 대상 파일입니다.
cohort_path = os.path.join(base_path, 'cohort_ver21_expanded_STEMI(2).csv')

print("데이터 클리닝 작업 시작 (24번째 열부터 삭제)")

try:
    # 2. 데이터 로드
    if not os.path.exists(cohort_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {cohort_path}")

    df_cohort = pd.read_csv(cohort_path)
    original_col_count = len(df_cohort.columns)
    print(f"원본 파일 로드 완료. (Shape: {df_cohort.shape}, Columns: {original_col_count})")

    # 3. 컬럼 삭제 (24번째 열 = 인덱스 23)
    if original_col_count > 23:
        cols_to_drop = df_cohort.columns[23:]
        print(f"삭제 대상 컬럼 ({len(cols_to_drop)}개): {list(cols_to_drop)}")

        df_cohort_cleaned = df_cohort.drop(columns=cols_to_drop)

        # 4. 동일한 파일명으로 덮어쓰기
        df_cohort_cleaned.to_csv(cohort_path, index=False, encoding='utf-8-sig')
        print(f"클리닝된 데이터 저장 완료: {cohort_path} (New Shape: {df_cohort_cleaned.shape})")

    else:
        print("컬럼 수가 23개 이하이므로, 삭제 작업을 건너뜁니다.")

except Exception as e:
    print(f"오류 발생: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
데이터 클리닝 작업 시작 (24번째 열부터 삭제)
원본 파일 로드 완료. (Shape: (1983, 38), Columns: 38)
삭제 대상 컬럼 (15개): ['study_id', 'report_0', 'report_1', 'rr_interval', 'qrs_onset', 'qrs_end', 't_end', 'heart_rate', 'qrs_duration', 'qt_interval', 'qtc', 'interpretation', 'text_stemi_flag', 'anchor_age', 'interp_lower']
클리닝된 데이터 저장 완료: /content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/cohort_ver21_expanded_STEMI(2).csv (New Shape: (1983, 23))


### 3. 최종 코호트 EDA